## Welcome to Lab 3 for Week 1 Day 4

Today we're going to build something with immediate value!

In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`

We're not going to use Tools just yet - we're going to add the tool tomorrow.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Looking up packages</h2>
            <span style="color:#00bfff;">In this lab, we're going to use the wonderful Gradio package for building quick UIs,
            and we're also going to use the popular PyPDF PDF reader. You can get guides to these packages by asking
            ChatGPT or Claude, and you find all open-source packages on the repository <a href="https://pypi.org">https://pypi.org</a>.
            </span>
        </td>
    </tr>
</table>

In [4]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [5]:
load_dotenv(override=True)
openai = OpenAI()

In [7]:
reader = PdfReader("/content/me/Sanjay.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [8]:
print(linkedin)

   
Contact
sanjayfuloria@gmail.com
www.linkedin.com/in/sanjayfuloria
(LinkedIn)
Top Skills
Unsupervised Learning
Applied Machine Learning
Linear Algebra
Certifications
Mathematics for Machine Learning
Programming for Everybody (Getting
Started with Python)
Capstone: Retrieving, Processing,
and Visualizing Data with Python
Machine Learning
Machine Learning Specialization
Sanjay Fuloria Ph.D.
Professor and Director Center for Distance and Online Education,
ICFAI FOUNDATION FOR HIGHER EDUCATION (a deemed to
be University under Section 3 of the UGC Act) , Hyderabad at IBS
Hyderabad
Hyderabad, Telangana, India
Summary
I have 26 years of experience in both academics and the corporate
world. I have handled marketing and sales, taught market research,
analytics and practiced business research, team management and
application of various analytics and machine learning tools and
techniques.
Experience
IBS Hyderabad
6 years 3 months
Professor and Director, Center for Distance and Online Education

In [9]:
with open("/content/me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [10]:
name = "Sanjay Fuloria"

In [11]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [12]:
system_prompt

"You are acting as Sanjay Fuloria. You are answering questions on Sanjay Fuloria's website, particularly questions related to Sanjay Fuloria's career, background, skills and experience. Your responsibility is to represent Sanjay Fuloria for interactions on the website as faithfully as possible. You are given a summary of Sanjay Fuloria's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nContact\nsanjayfuloria@gmail.com\nwww.linkedin.com/in/sanjayfuloria\n(LinkedIn)\nTop Skills\nUnsupervised Learning\nApplied Machine Learning\nLinear Algebra\nCertifications\nMathematics for Machine Learning\nProgramming for Everybody (Getting\nStarted with Python)\nCapstone: Retrieving, Processing,\nand Visualizing Data with Python\nMachine Learning\nMachine Learning Specialization\nSanjay Fuloria Ph.D.\nProfessor and

In [13]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

## Special note for people not using OpenAI

Some providers, like Groq, might give an error when you send your second message in the chat.

This is because Gradio shoves some extra fields into the history object. OpenAI doesn't mind; but some other models complain.

If this happens, the solution is to add this first line to the chat() function above. It cleans up the history variable:

```python
history = [{"role": h["role"], "content": h["content"]} for h in history]
```

You may need to add this in other chat() callback functions in the future, too.

In [14]:
gr.ChatInterface(chat, type="messages").launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7e9c5709bea0f169ee.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [15]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [16]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [17]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [18]:
import os
# The base_url should point to the OpenAI API endpoint
chatgpt = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url="https://api.openai.com/v1" # Corrected base_url
)

In [19]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = openai.chat.completions.parse(model="gpt-4o-mini", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [20]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content

In [21]:
reply

'I do not hold any patents. My expertise primarily lies in academia and corporate research, focusing on analytics and machine learning. If you have any questions about my work or experience in those areas, feel free to ask!'

In [22]:
evaluate(reply, "do you hold a patent?", messages[:1])

Evaluation(is_acceptable=True, feedback="The response is clear and directly answers the user's question about holding a patent. It also maintains a professional tone and invites further questions regarding the Agent's expertise, which is in line with the role of representing Sanjay Fuloria. Overall, the response is acceptable quality.")

In [23]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [24]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)

    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)
    return reply

In [25]:
gr.ChatInterface(chat, type="messages").launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e7a1a8cd39232eff24.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [1]:
%pip install python-dotenv openai pypdf gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 12.6 MB/s eta 0:00:00


In [2]:
%%writefile /content/.env
OPENAI_API_KEY = Your OpenAI API key
ANTHROPIC_API_KEY = Your Anthropic API key
GROQ_API_KEY = Your Groq API key

Writing /content/.env


In [3]:
!mkdir /content/me